In [ ]:
! pip install transformers datasets sentencepiece accelerate

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

os.environ["WANDB_DISABLED"] = "true"  # wandb 사용하지 않음

# 1. load pretrained tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [ ]:
# example input
inputs = tokenizer("I want to submit paper...", return_tensors="pt")

# initial inference for example input: might be random
outputs = model(**inputs.to(model.device))
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
labels = ["NEGATIVE", "POSITIVE"]
result = labels[torch.argmax(predictions)]
print(result)

In [ ]:
# 2. 데이터셋 불러오기
dataset = load_dataset("imdb")  # IMDb 영화 리뷰 데이터셋
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))  # 일부 데이터로 실습
test_dataset = dataset["test"].shuffle(seed=42).select(range(200))

# 3. 데이터 전처리
def preprocess_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

# 4. Trainer 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

In [ ]:
# 5. 훈련 실행
trainer.train()

# 6. 평가
results = trainer.evaluate()
print(results)


In [ ]:
# 모델 추론
inputs = tokenizer("I want to submit paper...", return_tensors="pt")
outputs = model(**inputs.to(model.device))
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
labels = ["NEGATIVE", "POSITIVE"]
result = labels[torch.argmax(predictions)]
print(result)